# Filtro de Detección de Spam

Se va a cosntruir un sistema de aprendizaje automático que pueda determinar si un correo electrónico se trata de un correo no deseado (SPAM) o si se treta de un correo legítimo.

Para entrenar el modelo se va a utlizar el siguiente conjunto de datos:

[2007 TREC Public Spam Corpus](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo07)

El corpus trec07p consta de 75419 mensajes divididos de la siguiente forma:
- 25220 ham
- 50199 spam

## Procesamiento de datos

En el archivo `full/index` tenemos una lista con la etiqueta de cada email (ham - spam) y la ubicación del correo completo. Dado que el conjunto de datos es muy amplio creamos una función que no sólo nos permita leer los correos con su con su correspondiente etiqueta sino también seleccionar un subconjunto de datos más manejable. 

La función `leer_correos` se encuentra dentro del módulo `read_emails` y devuelve una tupla con el subconjunto seleccionado con una lista con los cuerpos de los correos en la primera posición y otra lista con las etiquetas en la segunda.

### Procesamiento del texto HTML en los email

Vamos a echar un vistazo al aspecto de los correos almacenados:

In [4]:
from read_emails import leer_correos

X, y = leer_correos("email-dataset", 10)

print("-- Primer correo --")
print()
print("Etiqueta:", y[0])
print("Correo:")
print()
print(X[0])
print("-" * 100)

-- Primer correo --

Etiqueta: spam
Correo:

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bo

Como vemos el conjunto de datos de los correos está formado por cabecereas y campos adicionales, por lo que es necesario realizar tareas de preprocesamiento previo para que sean actos para ser ingeridos por el algoritmo de Machine Learning.

En primer lugar, debemos tratar las etiquetas HTML de los correos para ello nos servimos de la clase `HTMLParser` de la librería `html.parser` que implementamos a través de nuestra clase `HTMLStripper` que se encuentra en el módulo `html_stripper`. Veamos un ejemplo del funcionamiento

In [2]:
from html_stripper import HTMLStripper

html_parser = HTMLStripper()
html_parser.feed(X[0])

print(''.join(html_parser.data))

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: 
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: 
From: "Tomas Jacobs" 
Reply-To: "Tomas Jacobs" 
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit








Do you feel the pressure to perform and not rising to the occasion??





Try Viagra.....
your anxiety will be a thing of the past and you will
be back to your old self.



----8896484051606557286--




Ahora ya tenemos el correo libre de etiquetas HTML.

### Procesamiento del lenguaje natural

Por otro lado, después de eliminar las posible etiquetas HTML que contenga el correo electrónico, se deben procesar el texto para evitar el ruido innecesario que contiene el correo, esto es, signos de puntación, eliminar campos del correo que no son relevantes o eliminar los afijos de una palabra de forma que sólo se conserve la raiz de la misma, proceso que se conoce como Stemming.

Para realizar el preprocesamiento completo de los correos vamos a crear la clase ``EmailParser`` donde vamos a realizar todas la tareas necesarias en el siguiente orden:
1. Obtener el cuerpo del correo.
2. Eliminar las etiquetas HTML.
3. Eliminar urls.
4. Tokenizar el texto.
5. Limpiar el texto.

Ya tenemos implepentada la clase para eliminar las etiquetas HTML que incorporaremos a nuestra clase. Para eliminar las partes del correo que no nos interesan como la cabecera o el pie del correo electrónico nos valemos del paquete ``email`` para procesar los correos y quedarnos solamente con el cuerpo de los mismos.

Para tokenizar el texto y limpiar el texto (eliminar signos de punuacion, stopwords, Stemming) utilizamos el paquete ``nltk``. Se puede consulatar la implementación completa de la clase en el módulo ``email_parser``.

In [9]:
from email_parser import EmailParser

parser = EmailParser()

print("Correo original:")
print(X[1])

print("Correo limpiado:")
print(parser.parse(X[1]))

Correo original:
From bounce-debian-mirrors=ktwarwic=speedy.uwaterloo.ca@lists.debian.org  Sun Apr  8 13:09:29 2007
Return-Path: <bounce-debian-mirrors=ktwarwic=speedy.uwaterloo.ca@lists.debian.org>
Received: from murphy.debian.org (murphy.debian.org [70.103.162.31])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with ESMTP id l38H9S0I003031
	for <ktwarwic@speedy.uwaterloo.ca>; Sun, 8 Apr 2007 13:09:28 -0400
Received: from localhost (localhost [127.0.0.1])
	by murphy.debian.org (Postfix) with QMQP
	id 90C152E68E; Sun,  8 Apr 2007 12:09:05 -0500 (CDT)
Old-Return-Path: <yan.morin@savoirfairelinux.com>
X-Spam-Checker-Version: SpamAssassin 3.1.4 (2006-07-26) on murphy.debian.org
X-Spam-Level: 
X-Spam-Status: No, score=-1.1 required=4.0 tests=BAYES_05 autolearn=no 
	version=3.1.4
X-Original-To: debian-mirrors@lists.debian.org
Received: from xenon.savoirfairelinux.net (savoirfairelinux.net [199.243.85.90])
	by murphy.debian.org (Postfix) with ESMTP id 827432E3E5
	for <debian-mirrors@lists.debian.or

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
parser.parse(X[2])

'mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click authent viagra mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click'

## Codificación del conjunto de datos

Una vez realizado el preprocesamiento ya tenemos los correos electrónicos limpios de elementos innecesarios para la detección de SPAM, sin embargo, los correos siguen en formato de texto. Esto es un inconveninte dado que la mayoría de los algoritmos de Machine Learning no son capades de ingerir texto como parte del conjunto de datos. Por lo tanto, debemos realizar una serie de transformaciones para representar el texto de los email en un formato númerico.

Para codificar el texto utilizamos la clase ``CountVectorizer`` de la libreria de ``scikit-learn``

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

vectorizer.fit([X[0], X[2]])

CountVectorizer()

In [14]:
vectorizer.get_feature_names_out()

array(['00', '0000', '000000', '001', '00ffff', '0100', '0300', '04',
       '0400', '07', '08', '0px', '101', '12', '122', '129', '13', '144',
       '152', '17', '175', '186', '19', '1px', '2007', '200704081712',
       '202', '21', '211', '23', '24', '243', '2600', '32',
       '3b752b39332e', '41', '441b', '4619227b', '47e4', '48', '605',
       '74', '78', '7bit', '7stocknews', '80', '8859',
       '8896484051606557286', '89', '8f412ea9', '90', '97', '988', 'abef',
       'align', 'alternative', 'and', 'anxiety', 'aol', 'apr', 'at',
       'authentic', 'authenticv', 'back', 'be', 'bgcolor', 'body',
       'border', 'bottom', 'boundary', 'br', 'branded', 'by',
       'c94f920e24', 'ca', 'ccffaa', 'cellpadding', 'cellspacing',
       'center', 'charset', 'chkmail', 'cialis', 'clamav', 'clean',
       'click', 'color', 'com', 'content', 'crenshaw', 'date', 'discount',
       'div', 'do', 'dvgvhnrnvxvt', 'edt', 'encoding', 'ensmp', 'esmtp',
       'excoriationtuh', 'express', 'f05e057

In [15]:
X_vect = vectorizer.transform([X[2]])
X_vect.toarray()

array([[0, 1, 0, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 4, 1, 1, 7, 0, 0, 1, 1, 1,
        1, 0, 4, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2,
        3, 1, 2, 0, 1, 1, 2, 1, 0, 1, 0, 1, 0, 0, 0, 4, 1, 3, 1, 0, 0, 0,
        2, 0, 0, 1, 3, 0, 3, 1, 8, 0, 0, 0, 0, 2, 2, 0, 2, 1, 2, 0, 5, 6,
        1, 1, 4, 0, 1, 1, 1, 2, 1, 2, 0, 0, 4, 0, 0, 0, 2, 1, 4, 0, 0, 2,
        2, 1, 3, 3, 0, 4, 2, 2, 0, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        4, 2, 1, 0, 1, 1, 1, 2, 2, 0, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        2, 4, 0, 2, 1, 1, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2,
        2, 0, 1, 4, 0, 0, 2, 0, 2, 1, 0, 2, 1, 0, 0, 3, 2, 0, 3, 1, 8, 3,
        2, 2, 0, 0, 3, 2, 0, 0, 0, 1]], dtype=int64)

## Entrenamiento del algoritmo

Para comprobar que todo funcione correctamente vamos a considerar primero una pequeña muestra del conjunto de datos para no demorar demasiado tiempo.

Comenzamos cargando un subconjunto de 150 correos:

In [1]:
from read_emails import crear_dataset

X, y = crear_dataset('email-dataset', 150)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Parsing email: 150

Dividimos el conjunto de datos en un subconjunto de entrenamiento y otro de evaluación:

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [4]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [5]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [6]:
import pandas as pd

pd.DataFrame(X_train.toarray(), columns=[vectorizer.get_feature_names_out()])

,00,000,0000,000000,0001pt,000901c77a1f,0085,009,01,01000u,...,úíêãàææ½â,úóê¼þëñë,ü¼¼êõ,ýðåï,þu,þîñæ,þîñòµ¼,þîñôú,šè,ƒö
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Vamos a utlizar el algoritmo ``LogisticRegression`` de Sklearn. Se trata de un algoritmo de aprendizaje no supervisado.

In [7]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [8]:
y_pred = clf.predict(X_test)

In [9]:
print("\nPrediccion:\n", y_pred)
print("\nEtiquetas reales:\n", y_test)


Prediccion:
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'ham' 'ham' 'ham' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham']

Etiquetas reales:
 ['spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham']


In [10]:
from sklearn.metrics import accuracy_score

print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 0.933


## Aumentando el conjunto de datos

Ahora que ya hemos comprabado que tenemos una buena precisión con un conjunto pequeño de datos:

In [11]:
X, y = crear_dataset('email-dataset', 15000)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Parsing email: 15000

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [45]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [46]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [47]:
y_pred = clf.predict(X_test)

In [48]:
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.990


## Metricas 

NO usar:

In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Calcular las métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', average='micro', average='weighted')
recall = recall_score(y_test, y_pred, average='macro', average='micro', average='weighted')
f1 = f1_score(y_test, y_pred, average='macro', average='micro', average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Imprimir los resultados
print(f"Precisión (Accuracy): {accuracy:.4f}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Recall (Sensibilidad): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Matriz de Confusión:")
print(conf_matrix)
print("Reporte de Clasificación:")
print(class_report)

SyntaxError: keyword argument repeated: average (2070383821.py, line 5)

Usar:

In [60]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
import numpy as np
# Asegúrate de que las etiquetas sean de tipo texto ('ham' y 'spam')
print("Etiquetas en y_test:", np.unique(y_test))  # Deben ser ['ham', 'spam']
print("Etiquetas en y_pred:", np.unique(y_pred))  # Deben ser ['ham', 'spam']

# Calcular métricas con etiquetas de texto
precision = precision_score(y_test, y_pred, pos_label='spam')  # Aquí 'spam' es la clase positiva
recall = recall_score(y_test, y_pred, pos_label='spam')
f1 = f1_score(y_test, y_pred, pos_label='spam')

# Imprimir resultados
print(f"Precisión (Precision): {precision:.4f}")
print(f"Recall (Sensibilidad): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# También puedes usar un reporte de clasificación para ver más detalles
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)


Etiquetas en y_test: ['ham' 'spam']
Etiquetas en y_pred: ['ham' 'spam']
Precisión (Precision): 0.9927
Recall (Sensibilidad): 0.9939
F1-Score: 0.9933
Reporte de Clasificación:
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98      1080
        spam       0.99      0.99      0.99      3420

    accuracy                           0.99      4500
   macro avg       0.99      0.99      0.99      4500
weighted avg       0.99      0.99      0.99      4500

Matriz de Confusión:
[[1055   25]
 [  21 3399]]


# Conlusiones editar --------------------

¡Qué bien que haya funcionado! A continuación, te proporciono una interpretación detallada de los resultados que puedes incluir en tu informe, explicando cada métrica y la matriz de confusión:

---

### **Interpretación de los Resultados:**

#### **Métricas de Clasificación:**
- **Precisión (Precision): 0.9927**
  - La precisión indica que, de todas las predicciones de **spam** que el modelo hizo, el 99.27% de ellas fueron correctas. Es decir, el modelo tuvo una tasa muy baja de falsos positivos, lo que significa que muy pocos correos legítimos fueron clasificados como spam.

- **Recall (Sensibilidad): 0.9939**
  - El recall muestra que, de todos los correos que eran realmente **spam**, el 99.39% fueron correctamente identificados por el modelo. Este valor elevado indica que el modelo tiene un buen rendimiento al detectar el spam y minimizar los falsos negativos (cuando un correo de spam no se detecta como tal).

- **F1-Score: 0.9933**
  - El F1-score es la media armónica entre precisión y recall, lo que da una medida equilibrada entre ambas. Un valor de **0.9933** sugiere que el modelo tiene un rendimiento **muy alto**, capturando la mayoría de los correos spam sin marcar demasiados correos legítimos como spam.

#### **Reporte de Clasificación:**
El reporte de clasificación desglosa las métricas por clase:

- **Para la clase 'ham' (no spam):**
  - **Precisión**: 0.98, **Recall**: 0.98, **F1-Score**: 0.98.
  - Esto significa que el modelo tiene un rendimiento muy bueno para identificar correos legítimos. El 98% de los correos no spam fueron correctamente clasificados como **ham**, y solo un pequeño porcentaje de correos legítimos fueron incorrectamente clasificados como spam.

- **Para la clase 'spam':**
  - **Precisión**: 0.99, **Recall**: 0.99, **F1-Score**: 0.99.
  - El modelo tiene un rendimiento **excelente** en la clasificación de correos **spam**. Detectó el 99% de los correos de spam y casi no cometió errores al etiquetar los correos como spam.

#### **Accuracy (Precisión General): 0.99**
- El **accuracy** (precisión general) indica que el 99% de las predicciones del modelo fueron correctas. Este es un valor muy alto, lo que sugiere que el modelo tiene un rendimiento sobresaliente en general.

#### **Promedios:**
- **Macro promedio**: 0.99 para todas las métricas (precisión, recall, F1-score). Esto significa que el modelo tiene un rendimiento equilibrado entre las clases sin favorecer ninguna de ellas.
- **Weighted promedio**: 0.99 para todas las métricas. Este valor es ponderado por la cantidad de ejemplos en cada clase. Como las clases están relativamente balanceadas, el valor ponderado es similar al macro promedio.

---

### **Matriz de Confusión:**
La matriz de confusión es una herramienta importante para entender cómo el modelo está clasificando los correos en ambas clases (spam y ham):

```
[[1055   25]  -> 'ham' (no spam): 1055 correctos, 25 incorrectos (falsos positivos)
 [  21 3399]] -> 'spam': 21 incorrectos (falsos negativos), 3399 correctos
```

- **Verdaderos Positivos (TP)**: 3399 (correos spam correctamente clasificados como spam).
- **Verdaderos Negativos (TN)**: 1055 (correos no spam correctamente clasificados como no spam).
- **Falsos Positivos (FP)**: 25 (correos no spam incorrectamente clasificados como spam).
- **Falsos Negativos (FN)**: 21 (correos spam incorrectamente clasificados como no spam).

#### **Análisis de la Matriz de Confusión:**
- El modelo ha cometido **25 falsos positivos**, lo que significa que ha marcado 25 correos legítimos como spam. Esto podría causar que algunos correos importantes lleguen a la carpeta de spam.
- El modelo ha cometido **21 falsos negativos**, lo que significa que 21 correos de spam no fueron detectados y llegaron a la bandeja de entrada del usuario.

El hecho de que tanto los **falsos positivos** como los **falsos negativos** sean muy bajos indica que el modelo es muy efectivo para clasificar tanto correos legítimos como correos spam. 

### **Conclusión:**
El modelo de detección de spam ha mostrado un rendimiento excelente, con una **precisión** del 99%, una **sensibilidad (recall)** del 99% y un **F1-score** de 0.9933. Estos resultados indican que el modelo es capaz de identificar correctamente tanto correos legítimos como spam con una alta tasa de aciertos y muy pocos errores.

Si bien los **falsos positivos** (correos legítimos marcados como spam) y los **falsos negativos** (correos spam que no se detectan como tales) son mínimos, es importante seguir monitoreando el modelo y mejorar la clasificación en caso de que aumenten estos errores en un escenario del mundo real.

---

¡Espero que esta interpretación te sea útil para tu informe! Si tienes alguna pregunta o necesitas más detalles, no dudes en preguntar.